# ML-BPMN Getting Started with scikit-learn, PMML and Camunda

*... a tutorial for students in the FHNW, written by [Andreas Martin, PhD](https://andreasmartin.ch).*

|[![deepnote](https://deepnote.com/buttons/launch-in-deepnote-small.svg)](https://deepnote.com/launch?url=https%3A%2F%2Fgithub.com%2FAI4BP%2Fainotes%2Fblob%2Fmain%2Fipynb%2Fexpense-authorization-process%2Fexpense-authorization.ipynb)|[![Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AI4BP/ainotes/blob/main/ipynb/expense-authorization-process/expense-authorization.ipynb)|[![Gitpod](https://img.shields.io/badge/Gitpod-Run%20in%20VS%20Code-908a85?logo=gitpod)](https://gitpod.io/#https://github.com/AI4BP/ainotes/)|[![GitHub.dev](https://img.shields.io/badge/github.dev-Open%20in%20VS%20Code-908a85?logo=github)](https://github.dev/AI4BP/ainotes/blob/main/ipynb/expense-authorization-process/expense-authorization.ipynb)|
|-|-|-|-|

This short tutorial is intended to provide a straight forward introduction to machine learning using the widely used Python library **scikit-learn** (aka sklearn).

> Trivia: The name *SciKit* is derived from its original intention being a SciPy Toolkit. SciPy is another Python library for scientific computing.

Sklearn enjoys huge popularity when it comes to classic machine learning methods; it is well documented, has a large developer community and besides the official documentation there are plenty of other good resources for the ML toolkit available on the web.

> Sklearn is intended for **classical ML** and **not for Deep Learning**, although a Multi-layer Perceptron (MLP), for example, can be trained. Since Sklearn does **not support GPUs**, it is not suitable for large-scale applications.

## Data and Use Case
This tutorial uses historical data from an expense reporting and audit process — the **expense authorization process** is depicted in the following Fig a:

![](https://github.com/AI4BP/ainotes/raw/main/sklearn-getting-started/ipynb/images/expense-authorization-sklearn.png)

**Fig a**: expense authorization process

This (possibly synthetic) data was collected by humans, which is approved or not based on the expense **category**, **urgency**, **target price** and actual **price** paid.

> This use case has been inspired by an example/article of Donato Marrazzo (Red Hat, Inc.). He provided data, in a related GitHub repository [[1]](https://github.com/dmarrazzo/rhdm-dmn-pmml-order), of an **expense approval process**, which is used here in this tutorial, along with a series of articles ([[2]](https://developers.redhat.com/blog/2021/01/14/knowledge-meets-machine-learning-for-smarter-decisions-part-1#conclusion) and [[3]](https://developers.redhat.com/blog/2021/01/22/knowledge-meets-machine-learning-for-smarter-decisions-part-2#conclusion)).

### 🚧 Main Task
The task in this tutorial is to train an ML model step by step and then generate a PMML file, which then implements the `Approve expense order (PMML)` activity depicted in Fig a.

## 0. Initialization Configuration
In the following there is some code for initialization. For example, the URL to the data `url_data` and the BPMN/DMN models `url_modelling` is set.

In [ ]:
import os

url_github = "https://raw.githubusercontent.com/AI4BP/ainotes/main"
project_folder = "sklearn-getting-started"
working_dir = os.path.normpath(os.getcwd()+"/../")
url_data = "data"
url_data = f"{working_dir}/{url_data}" if os.path.exists(f"{working_dir}/{url_data}") else f"{url_github}/{project_folder}/{url_data}"
print(url_data)
url_modelling = "modelling"
url_modelling = f"{working_dir}/{url_modelling}" if os.path.exists(f"{working_dir}/{url_modelling}") else f"{url_github}/{project_folder}/{url_modelling}"
print(url_modelling)

## 1. Load the CSV File
Load the CSV file from GitHub and feed the data into the *data* variable by using [pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html). Pandas is intended to be a data analysis and manipulation tool, which is used here and the following steps until dataset separation.

In [ ]:
import pandas

data = pandas.read_csv(f"{url_data}/expense-authorization.csv", sep=",")

data

## 10. PMML Export
If necessary, the `sklearn2pmml` must be organised in advance via pip. Since `sklearn2pmml` requires the JRE (Java Runtime Environment), JRE may have to be installed beforehand, depending on the Jupyter environment.

In [ ]:
try:
    import sklearn2pmml
except:
    !pip -qq install sklearn2pmml

from sklearn2pmml import _java_version

if _java_version("UTF-8") is None:
    !sudo apt -qq update > /dev/null
    !sudo apt -qq install -y default-jre > /dev/null

print(_java_version("UTF-8"))

Finally, the trained model can now be exported with the library [sklearn2pmml](https://github.com/jpmml/sklearn2pmml) to PMML.



In [ ]:
import sklearn2pmml

pmml_file_name = "expense-authorization-sklearn.pmml"

from sklearn2pmml import make_pmml_pipeline, sklearn2pmml

pipeline = make_pmml_pipeline(
    model,
    active_fields=["category", "urgency", "targetPrice", "price"],
    target_fields=["approved"],
)
sklearn2pmml(pipeline, pmml_file_name)

## 11. PMML Deployment
After we have created the PMML file, we can upload it with the follwing code.

> You may need to change the `tenant-id` first.

In [ ]:
import requests

camunda_eninge_rest = "https://digibp.herokuapp.com/engine-rest/deployment/create"

request_file = {pmml_file_name: (pmml_file_name, open(pmml_file_name, "rb"))}

request_data = {
    "tenant-id": "showcase",  # please change the tenant-id
}

response = requests.post(camunda_eninge_rest, files=request_file, data=request_data)
deployment_id = response.json()["id"]

print(deployment_id)

## 12. PMML Testing
For executing the PMML files together with a Camunda Workflow, we can use the provided classroom instantiation, which has been extended with `jpmml`, the [Java PMML API](https://github.com/jpmml). Now can use the obtained `deployment-id` to construct the `requests` url - **so please change the following** `deployment_id`.

In [ ]:
try:
    deployment_id
except:
    deployment_id = (
        "f85423d8-47e1-11ec-834e-eea2248ab9a4"  # please change the deployment-id!
    )

evaluate_url = (
    f"https://digibp.herokuapp.com/pmml/{deployment_id}/{pmml_file_name}/evaluate"
)

print(evaluate_url)

Now we are ready to send a `GET` request to the DigiBP PMML API to retrieve our input variable structure of our deployed PMML file.

In [ ]:
import requests, json

response = requests.get(f"{evaluate_url}/generate-input")

print(json.dumps(response.json(), indent=2))

Finally, in this step, we can copy-and-paste the input variable structure from above as input to the `payload` variable and adjust the values.

In [ ]:
payload = {"variables": {"targetPrice": 520, "urgency": 0, "price": 480, "category": 1}}

response = requests.post(evaluate_url, json=payload)

print(json.dumps(response.json(), indent=2))

We now should have received a possible output (prediction) of the PMML evaluation. In this use case, we should receive an `approved` variable, which is of type `Boolean`.

### 🔀 Alternative Way
This step is an alternative approach by using a Swagger UI. The provided classroom instantiation provides an own basic testing Swagger UI. This [PMML API Swagger UI](https://digibp.herokuapp.com/swagger-ui/#/pmmlapi) gives us the possibility, to `generate-input` fields and `evaluate` our PMML model as depicted in Fig 12.

![](https://github.com/AI4BP/ainotes/raw/main/sklearn-getting-started/ipynb/images/camunda-pmml-api.png)

**Fig 12**: PMML API Swagger UI

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d0fd9439-c605-456d-a503-a84c619fa8f8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>